In [143]:
import pandas as pd
import numpy as np
import json

In [144]:
df = pd.read_csv('data/flipkart/flipkart.csv')

In [145]:
df.drop(['product_url', 'pid', 'crawl_timestamp', 'discounted_price', 'is_FK_Advantage_product', 'overall_rating'], inplace=True, axis=1)
df.rename(columns={'uniq_id': 'id', 'product_name': 'name', 'product_category_tree': 'category', 'retail_price': 'price', 'product_rating': 'rating'}, inplace=True)

In [146]:
def parseProductSpecifications(row):
    product_specification = row.product_specifications
    try:
        product_specification = product_specification.replace('{"product_specification"=>', '')
        product_specification = product_specification[0:-1]
        product_specification = product_specification.replace('=>', ':')
        product_specification = eval(product_specification) #string to list

        specifications = {}
        aditional = []
        for ps in product_specification:
            value = ps['value']
            if 'key' in ps.keys():
                key = ps['key']
                specifications[key] = value
            else:
                aditional.append(value)     
        if len(aditional) != 0:
            specifications['Aditional information'] = '; '.join(aditional)            
            
        return specifications
    except:
        return {}
    

df['product_specifications'] = df.apply(parseProductSpecifications, axis=1)

In [147]:
def parseCategory(row):
    cat = row.category[2:-2]
    cat = cat.split(' >> ')
    return cat

df['category'] = df.apply(parseCategory, axis=1)

In [148]:
def replaceRating(row):
    if row.rating == 'No rating available':
        return np.round(np.random.uniform(0.1, 5.0), 1)
    return float(row.rating)
    
df['rating'] = df.apply(replaceRating, axis=1)

In [149]:
def parseImage(row):
    try:
        image = eval(row.image)
        if len(image) > 0:
            return image[0]
        return ""

    except:
        return ""


df['image'] = df.apply(parseImage, axis=1)

In [150]:
df['stock'] = np.random.randint(0, 300, df.shape[0])
df['store'] = 'flipkart'

In [151]:
df.to_csv('data/flipkart/products.csv', encoding='utf-8', index=False)

In [ ]:
result = df.to_json(orient="records", default_handler=str)
parsed = json.loads(result)
json_string = json.dumps(parsed, indent=4)
with open('data/flipkart/products.json', 'w') as outfile:
    outfile.write(json_string)